# Summary
This document corresponds to Exercise 10 of [this file](https://github.com/PerformanceEstimation/Learning-Performance-Estimation/blob/main/Course.pdf).

If [PEPit](https://pypi.org/project/PEPit/) is not already installed, please execute the following cell.

In [ ]:
!pip install pepit

### Exercise 10.1

No code required.

### Exercise 10.2

The following cell allows to compute the desired worst-case ratio.

In [ ]:
from PEPit import PEP
from PEPit.functions import SmoothStronglyConvexFunction
from PEPit.functions import ConvexFunction
from PEPit.primitive_steps import proximal_step


def wc_douglas_rachford_splitting_contraction(mu, L, alpha, theta, verbose=1):
    
    # Instantiate PEP
    problem = PEP()

    # Declare a convex and a smooth strongly convex function.
    f = problem.declare_function(SmoothStronglyConvexFunction, mu=mu, L=L)
    h = problem.declare_function(ConvexFunction)

    # Define the function to optimize as the sum of func1 and func2
    F = f + h

    # Start by defining its unique optimal point xs = x_*
    xs = F.stationary_point()

    # Then define the starting points w0 and w0p of the algorithm
    w0 = problem.set_initial_point()
    w0p = problem.set_initial_point()

    # Set the initial constraint that is the distance between w0 and w0p
    problem.set_initial_condition((w0 - w0p) ** 2 <= 1)

    # Compute n steps of the Douglas Rachford Splitting starting from w0
    x0, _, _ = proximal_step(w0, h, alpha)
    y0, _, _ = proximal_step(2 * x0 - w0, f, alpha)
    w1 = w0 + theta * (y0 - x0)

    # Compute n steps of the Douglas Rachford Splitting starting from w0p
    x0p, _, _ = proximal_step(w0p, h, alpha)
    y0p, _, _ = proximal_step(2 * x0p - w0p, f, alpha)
    w1p = w0p + theta * (y0p - x0p)

    # Set the performance metric to the final distance between w and wp
    problem.set_performance_metric((w1 - w1p) ** 2)

    # Solve the PEP
    pepit_tau = problem.solve(verbose=verbose)

    # Return the worst-case guarantee of the evaluated method 
    return pepit_tau

Numerical trials for a few values of $(\mu,L,\alpha)$ (and $\theta=1$):

In [ ]:
mu = .1
L = 1
alpha = 1
theta = 1

wc_douglas_rachford_splitting_contraction(mu, L, alpha, theta,  verbose=1)

(max(1 / (1 + mu * alpha), alpha * L / (1 + alpha * L))) ** 2

### Exercise 10.3

Low-dimensional worst-case examples?

In [ ]:
from PEPit import PEP
from PEPit.functions import SmoothStronglyConvexFunction
from PEPit.functions import ConvexFunction
from PEPit.primitive_steps import proximal_step


def wc_douglas_rachford_splitting_contraction_lowdim(mu, L, alpha, theta, n, verbose=1):
    
    # Instantiate PEP
    problem = PEP()

    # Declare a convex and a smooth strongly convex function.
    f = problem.declare_function(SmoothStronglyConvexFunction, mu=mu, L=L)
    h = problem.declare_function(ConvexFunction)

    # Define the function to optimize as the sum of func1 and func2
    F = f + h

    # Start by defining its unique optimal point xs = x_*
    xs = F.stationary_point()

    # Then define the starting points w0 and w0p of the algorithm
    w0 = problem.set_initial_point()
    w0p = problem.set_initial_point()

    # Set the initial constraint that is the distance between w0 and w0p
    problem.set_initial_condition((w0 - w0p) ** 2 <= 1)

    # Compute n steps of the Douglas Rachford Splitting starting from w0
    w = w0
    for _ in range(n):
        x, _, hx = proximal_step(w, h, alpha)
        y, _, fy = proximal_step(2 * x - w, f, alpha)
        w = w + theta * (y - x)

    # Compute n steps of the Douglas Rachford Splitting starting from w0p
    wp = w0p
    for _ in range(n):
        xp, _, hxp = proximal_step(wp, h, alpha)
        yp, _, fyp = proximal_step(2 * xp - wp, f, alpha)
        wp = wp + theta * (yp - xp)

    # Set the performance metric to the final distance between w and wp
    problem.set_performance_metric((w - wp) ** 2)

    # Solve the PEP
    pepit_tau = problem.solve(verbose=verbose, dimension_reduction_heuristic="trace")
        
    # Return the worst-case guarantee of the evaluated method 
    return pepit_tau

In [ ]:
mu = .1
L = 1
alpha = 10
theta = 1
n = 10

pepit_tau = wc_douglas_rachford_splitting_contraction_lowdim(mu, L, alpha, theta, n, verbose=1)
